In [ ]:
!pip install torch torchvision --quiet
!pip install transformers  --quiet
!pip install pandas  --quiet
!pip install numpy  --quiet
!pip install sentencepiece  --quiet
!pip install sentence-splitter  --quiet
!pip install shap --quiet





     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 57.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 25.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 103.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 76.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.0/45.0 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.9/547.9 kB 9.8 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# **Imports**

In [ ]:
import random
import torch
import numpy as np
import pandas as pd
from tqdm import tqdm
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AdamW,AutoModelForQuestionAnswering, get_linear_schedule_with_warmup, get_cosine_schedule_with_warmup
from transformers import DebertaTokenizer, DebertaModel, BartTokenizer
import math

# **Model loading**

In [ ]:
# Use a GPU if you have one available (Runtime -> Change runtime type -> GPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Set seeds for reproducibility
random.seed(26)
np.random.seed(26)
torch.manual_seed(26)

tokenizer = AutoTokenizer.from_pretrained("EMBO/BioMegatron345mUncased", do_lower_case=True)

model = AutoModelForSequenceClassification.from_pretrained("EMBO/BioMegatron345mUncased")
model.to(device) # Send the model to the GPU if we have one


Some weights of MegatronBertForSequenceClassification were not initialized from the model checkpoint at EMBO/BioMegatron345mUncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


MegatronBertForSequenceClassification(
  (bert): MegatronBertModel(
    (embeddings): MegatronBertEmbeddings(
      (word_embeddings): Embedding(30592, 1024, padding_idx=0)
      (position_embeddings): Embedding(512, 1024)
      (token_type_embeddings): Embedding(2, 1024)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): MegatronBertEncoder(
      (layer): ModuleList(
        (0-23): 24 x MegatronBertLayer(
          (attention): MegatronBertAttention(
            (ln): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
            (self): MegatronBertSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): MegatronBertSelfOutput(
              (dense): Linear(in_features=1024, out_featur

In [ ]:
import pandas as pd

df=pd.read_csv('./new_data_mutagenicity.csv')

# Apply the function to the column
df['label'] = df['label'].apply(lambda x: x.startswith("['True"))
#df['label'] = df['label'].apply(lambda x: x[0] == "['False - AMES non Mutagenic']")

new_column_names = {'text': 'Abstract', 'label': 'AMES'}
df.rename(columns=new_column_names, inplace=True)


# **Prediction**

In [ ]:
def predict(passage,question):
  sequence = tokenizer.encode_plus(passage,question, return_tensors="pt")['input_ids'].to(device)

  logits = model(sequence)[0]
  probabilities = torch.softmax(logits, dim=1).detach().cpu().tolist()[0]
  proba_yes = round(probabilities[1], 2)
  proba_no = round(probabilities[0], 2)

  print(f"Question: {question}, Yes: {proba_yes}, No: {proba_no}")

  if (proba_yes >= proba_no):
    return True
  else:
    return False






In [ ]:
filter=list(df.index)

In [ ]:
preds=[]
index=[]
for i in filter:
    try:
      passage=df.Abstract[i]
      question =df.questions[i]
      answer=predict(passage,question)
      preds.append(answer)
      index.append(i)
    except Exception as e:
      print(e)
      continue



Question: Is AZS mutagenic for TA98 or TA100 strain?, Yes: 0.01, No: 0.99
The size of tensor a (515) must match the size of tensor b (512) at non-singleton dimension 1
Question: Is 1-NP mutagenic for TA98 strain?, Yes: 0.89, No: 0.11
Question: Is 1-NP mutagenic for TA98 or TA100 or TA1535 strain?, Yes: 0.92, No: 0.08
Question: Is 1-NP mutagenic for TA98 strain?, Yes: 0.85, No: 0.15
Question: Is 1-NP mutagenic for TA98 strain?, Yes: 0.09, No: 0.91
The size of tensor a (534) must match the size of tensor b (512) at non-singleton dimension 1
Question: Is 1-NP mutagenic for TA98 strain?, Yes: 0.1, No: 0.9
Question: Is 1-NP mutagenic for TA98 strain?, Yes: 0.86, No: 0.14
Question: Is 1-NP mutagenic for TA98 strain?, Yes: 0.19, No: 0.81
Question: Is 1-NP mutagenic for TA98 strain?, Yes: 0.07, No: 0.93
The size of tensor a (558) must match the size of tensor b (512) at non-singleton dimension 1
Question: Is 1-NP mutagenic for TA98 or TA100 strain?, Yes: 0.88, No: 0.12
Question: Is 1-NP mutage

In [ ]:
len(index)


1260

In [ ]:
df=df[df.index.isin(index)]
true_results=df['AMES'].tolist()

In [ ]:
#With Deberta model w no maintenance of equal samples of the two classes for 3 epochs
from sklearn.metrics import classification_report
print(classification_report(true_results, preds,output_dict=True))

{'False': {'precision': 0.4446529080675422, 'recall': 0.41725352112676056, 'f1-score': 0.4305177111716621, 'support': 568}, 'True': {'precision': 0.5447042640990372, 'recall': 0.5722543352601156, 'f1-score': 0.5581395348837209, 'support': 692}, 'accuracy': 0.5023809523809524, 'macro avg': {'precision': 0.4946785860832897, 'recall': 0.4947539281934381, 'f1-score': 0.49432862302769154, 'support': 1260}, 'weighted avg': {'precision': 0.4996017480467442, 'recall': 0.5023809523809524, 'f1-score': 0.5006084270516182, 'support': 1260}}


In [ ]:
#With Deberta model w no maintenance of equal samples of the two classes for 3 epochs
from sklearn.metrics import classification_report
print(classification_report(true_results, preds))

              precision    recall  f1-score   support

       False       0.44      0.42      0.43       568
        True       0.54      0.57      0.56       692

    accuracy                           0.50      1260
   macro avg       0.49      0.49      0.49      1260
weighted avg       0.50      0.50      0.50      1260

